Import keras API

In [0]:
from keras import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding, LSTM, Dense, Input, Bidirectional, RepeatVector, Concatenate, Activation, Dropout
from keras.activations import softmax
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from keras.applications.inception_v3 import InceptionV3

from keras.optimizers import Adam

Using TensorFlow backend.


*Import twitter API & create app object*

In [0]:
import numpy as np
import tweepy
consumer_key = "JLBMyDbzgUqb5MxBUOdXRMfSM"
consumer_secret = "C01WULLbjHpYWxWwisOz5CnHCFzDF5lxNSPwgYYkQPxJN0aohm"
access_token = "848193493-hBvelUojIMU2LdhpDkJfRhecVmcwk1Wtqu9uLNgL"
access_token_secret = "DEab2D6pfdXmzPqvSMWrIIkOROaW8wXuXpWtZ5vxTSqqf"

In [0]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

# Get tweets, do some preprocessing, and save

Mount drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


I'm not really going to do any preprocessing for now... all I have done is put 
everything in lowercase and split on spaces. I have also removed retweets. Return to this later. Numpy array saved to parsed_tweets.npy

In [0]:
#the number of items to read
NUM_ITEMS = 10000


def process_status(status):
    #prune retweets
    if status.retweeted:
        return

    #split on spaces, to lowercase
    words = status.text.lower().split()

    #add start and end tags
    words = ['<START>'] + words + ['<END>']
    
    #for some reason some of these retweets are not tagged as retweets??
    if words[1] == 'rt':
        return
    #add to tweet list
    tweet_list.append(words)

if __name__=="__main__":
    tweet_list = []
    for status in tweepy.Cursor(api.user_timeline).items(NUM_ITEMS):
      process_status(status)
    #save in numpy array
    np_tweet_list = np.array(tweet_list)
    np.save('gdrive/My Drive/tweetbot/parsed_tweets.npy', np_tweet_list)

# Creat word indices

In [0]:
parsed_tweets = np.load('drive/My Drive/tweetbot/parsed_tweets.npy', allow_pickle=True)
vocab = set()
for tweet in parsed_tweets:
  for word in tweet:
    vocab.add(word)
vocab = list(vocab)
vocab.sort()
id_to_word = {}
#we are not including 0 as an index.
for i in range(0,len(vocab)):
  id_to_word[i+1] = vocab[i]

word_to_id = {}
for key, value in id_to_word.items():
  word_to_id[value] = key
assert(len(id_to_word) == len(word_to_id))

VOCAB_SIZE = len(vocab)


Test Word Indices

# Creating an Input Generator

In [0]:
def training_generator(batch_size=128):
  #global input, output lists
  input_list = []
  output_list = []
  example_count = 0
  VOCAB_SIZE = len(word_to_id)
  MAX_LEN = 40 #this is subject to change -- we want within 40 chars
  while True:
    for tweet in tweet_list:
      tweet_ids = []
      for word in tweet:
        tweet_ids.append(word_to_id[word])
      for slice_index in range(1,len(tweet)):
        example_count +=1
        #generate input
        input = tweet_ids[:slice_index]
        input = input + [0]*(MAX_LEN - len(input)) #pad with zeros

        #generate output
        output= [0]*(VOCAB_SIZE+1)
        target_word_id = tweet_ids[slice_index]
        if target_word_id not in range(len(output)):
          print(target_word_id)
          print(len(output))
          print(VOCAB_SIZE)
          print(len(word_to_id))
          continue
        output[target_word_id] = 1
        #print(target_word_id)
        #add to global lists
        input_list.append(input)
        output_list.append(output)
        #return a batch of generated input/output
        if example_count % batch_size == 0:
          pair = (np.array(input_list), np.array(output_list))
          assert len(input_list) == len(output_list)
          assert len(input_list) == batch_size
          yield pair
          input_list = []
          output_list = []
        

test the generator

In [0]:
batch_size = 10
generator = training_generator(batch_size)
results= next(generator)
input, output = results
for row in input:
  for word in row:
    if word in id_to_word:
      print(id_to_word[word])
  print('\n')

<START>


<START>
@nctsmtown_127


<START>
@nctsmtown_127
that


<START>
@nctsmtown_127
that
hoodie


<START>
@nctsmtown_127
that
hoodie
looks


<START>
@nctsmtown_127
that
hoodie
looks
so


<START>
@nctsmtown_127
that
hoodie
looks
so
soft


<START>


<START>
@fckyeahcharli


<START>
@fckyeahcharli
already




#Creating a Model!
We will be using a bidirectional lstm with a word encoding layer.


In [0]:
MAX_LEN = 40
EMBEDDING_DIM=300
vocab_size = len(word_to_id) + 1

# Text input
text_input = Input(shape=(MAX_LEN,))
embedding = Embedding(vocab_size, EMBEDDING_DIM, input_length=MAX_LEN)(text_input)
x = Bidirectional(LSTM(128, return_sequences=False))(embedding)
pred = Dense(vocab_size, activation='softmax')(x)
model = Model(inputs=[text_input],outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 40, 300)           1231800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_1 (Dense)              (None, 4106)              1055242   
Total params: 2,726,338
Trainable params: 2,726,338
Non-trainable params: 0
_________________________________________________________________


#Training the Model
We will use fit_generator to train the model.

In [0]:
#toggle this

batch_size = 128
generator = training_generator(batch_size)
steps = len(tweet_list) * MAX_LEN // batch_size 

weight_filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(weight_filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
callbacks=callbacks_list
model.fit_generator(generator, steps_per_epoch=len(tweet_list) * MAX_LEN // batch_size, verbose=True, epochs=10)

Epoch 1/10
524/524 [==============================] - 98s 188ms/step - loss: 4.0082 - accuracy: 0.3256
Epoch 2/10
524/524 [==============================] - 98s 187ms/step - loss: 3.6834 - accuracy: 0.3755
Epoch 3/10
524/524 [==============================] - 100s 191ms/step - loss: 3.4004 - accuracy: 0.4290
Epoch 4/10
231/524 [============>.................] - ETA: 54s - loss: 3.1219 - accuracy: 0.4739

KeyboardInterrupt: ignored

save the model

In [0]:
model.save_weights("gdrive/My Drive/tweetbot/model.h5")

load the model

In [0]:
model.load_weights("drive/My Drive/tweetbot/model.h5")

#Decoder
Greedy decoder here

In [0]:
def decoder():
  sentence = ['<START>']
  sentence_nums = [word_to_id['<START>']]
  end_reached = False
  while end_reached == False:
    input = np.array([sentence_nums+(40-len(sentence_nums))*[0.0]])
    output = model.predict(input)
    output = output[0]
    max_ind = np.argmax(output, axis = 0)
    token = id_to_word[max_ind]
    sentence.append(token)
    sentence_nums.append(max_ind)
    if token == '<END>':
      end_reached = True
  return sentence

In [0]:
print(decoder())

['<START>', 'i', 'want', 'to', 'see', 'taemin', '<END>']


#Beam Decoder

In [0]:
def beam_decoder(n):
  sequence = [word_to_id['<START>']] #sequences stored as lists of word ids
  sequence_tuples = [(1,sequence)] #tuples of (probability, sequence)
  end_reached = False
  for i in range(MAX_LEN):
    candidates = [] #will store the n*n candidates generated by this iteration
    #iterate through n sequences
    for sequence_tuple in sequence_tuples:
      prob, sequence = sequence_tuple
      input = np.array([sequence+(40-len(sequence))*[0.0]])
      output = model.predict(input)
      output = output[0]
      n_largest_inds = (-output).argsort()[:n]
      #for each sequence, generate n new candidate sequences
      for ind in n_largest_inds:
        cand_seq = sequence + [ind]
        cand_prob = prob * output[ind]
        candidate = (cand_prob, cand_seq)
        candidates.append(candidate)
    #prune the candidates and repeat
    candidates.sort(reverse = True)
    if len(candidates) >n:
      candidates = candidates[:n]
    sequence_tuples = candidates
  
  #Now that we have reached max_len, output the most likely candidate
  max_prob, max_sequence = max(sequence_tuples)
  sentence = []
  for id in max_sequence:
    sentence.append(id_to_word[id])
    if id == word_to_id['<END>']:
      break
  return sentence

In [0]:
beam_decoder(2)

['<START>', 'i', 'want', 'to', 'see', 'taemin', '<END>']

# Sample Decoder

In [0]:
def sample_decoder():
  sentence = ['<START>']
  sentence_nums = [word_to_id['<START>']]
  end_reached = False
  while end_reached == False:
    input = np.array([sentence_nums+(40-len(sentence_nums))*[0.0]])
    output = model.predict(input)
    output = output[0]
    output = output/(1.00001)
    random_word = np.random.multinomial(1, output, size=1)
    max_ind = np.argmax(random_word[0], axis = 0)
    token = id_to_word[max_ind]
    sentence.append(token)
    sentence_nums.append(max_ind)
    if token == '<END>':
      end_reached = True
  return sentence

In [0]:
for i in range(10): 
    print(sample_decoder())

['<START>', 'didn’t', 'valley', 'ate', '<END>']
['<START>', '@edw1ndrood', 'great', '<END>']
['<START>', 'https://t.co/rcqljlwawd', 'https://t.co/l7ayujxjyw', '<END>']
['<START>', 'i’m', 'can’t', 'scared', 'to', 'bridge', 'with', 'saving', 'dark:', 'robyn', 'square', 'wear', 'which', 'end', 'are', 'https://t.co/r02rjl1gm8', '<END>']
['<START>', 'yummy', 'is', '....', '<END>']
['<START>', 'this', 'girl', '🕯', 'literally', 'done', 'fact', 'for', 'all', 'your', 'jepsen', 'https://t.co/6ylev2gj00', '<END>']
['<START>', 'talking', 'https://t.co/id5xo8wmbb', '<END>']
['<START>', 'when', 'taemin', 'said', 'https://t.co/boioi5li2i', '<END>']
['<START>', 'zoo', 'can’t', 'ate', 'https://t.co/vnlj4gxgez', '<END>']
['<START>', 'carly', 'mood', 'with', 'right', 'such', 'good', 'omg', '<END>']


# Semi-greedy decoder

In [0]:
  def semi_beam(n):
    #Get a random first word.
    sentence = ['<START>']
    sentence_nums = [word_to_id['<START>']]
    input = np.array([sentence_nums+(40-len(sentence_nums))*[0.0]])
    output = model.predict(input)
    output = output[0]
    output = output/(1.00001)
    random_word = np.random.multinomial(1, output, size=1)
    max_ind = np.argmax(random_word[0], axis = 0)
    token = id_to_word[max_ind]
    sentence.append(token)
    sentence_nums.append(max_ind)
    #greedily decode from there
    return beam_decoder(n,sentence_nums)


  def beam_decoder(n, sequence):
    sequence_tuples = [(1,sequence)] #tuples of (probability, sequence)
    end_reached = False
    for i in range(1,MAX_LEN):
      candidates = [] #will store the n*n candidates generated by this iteration
      #iterate through n sequences
      for sequence_tuple in sequence_tuples:
        prob, sequence = sequence_tuple
        input = np.array([sequence+(40-len(sequence))*[0.0]])
        output = model.predict(input)
        output = output[0]
        n_largest_inds = (-output).argsort()[:n]
        #for each sequence, generate n new candidate sequences
        for ind in n_largest_inds:
          cand_seq = sequence + [ind]
          cand_prob = prob * output[ind]
          candidate = (cand_prob, cand_seq)
          candidates.append(candidate)
      #prune the candidates and repeat
      candidates.sort(reverse = True)
      if len(candidates) >n:
        candidates = candidates[:n]
      sequence_tuples = candidates
    
    #Now that we have reached max_len, output the most likely candidate
    max_prob, max_sequence = max(sequence_tuples)
    sentence = []
    for id in max_sequence:
      sentence.append(id_to_word[id])
      if id == word_to_id['<END>']:
        break
    return sentence

In [0]:
if __name__=="__main__":
  predictions = set()
  for i in range(50):
    #print('.')
    output = semi_beam(3)
    output = output[1:]
    output = output[:-1]
    #print(output)
    prediction = ''
    for word in output:
      prediction += word + ' '
    #print(prediction)
    predictions.add(prediction)


  for prediction in predictions:
    print(prediction)

“i want to see taemin 
robyn was the best video of pop in time 
@ncutigatwa this is my best song on the life 
https://t.co/uwzpy3eyzg 
we love a stan 
@edw1ndrood we talk about sure 
dressed up as as halloween as it lmao 
still thinking about this video lol 
@spencergbruce i’m jealous 
i want to see taemin 
on going to see taemin 
literally hate my life 
i’m going to go see charli xcx concert 
tossed omg that my life 
phone is my life 
thinking about taemin ... also at the charli xcx concert concert 
https://t.co/qlcltq2wio 
bay need to see taemin 
stop need to see taemin 
white thinking about taemin 
won’t taemin said taemin 
professor: @spencergbruce i was just to see robyn 
@charli_xcx robyn 
@crispiesthand hate my life 
#1 was a stan 
we’re need to see the charli xcx concert on time but it 💀 
mitchell. ate omg omg 
@arca1000000 ate omg 
😍 hate 
@musicnewsfact robyn ... taste 
https://t.co/ovt9zef21x yes 
hello need to see taemin 
2 xcx is my life 
if you need to see taemin 
u yes 


In [0]:
for i in range(10):
  print(semi_beam(3))

['<START>', 'joni', 'putting', 'on', 'my', 'life', '<END>']
['<START>', 'the', 'way', 'that', 'does', 'not', 'have', 'to', 'go', 'to', 'go', 'to', 'go', 'only', 'go', 'me', '<END>']
['<START>', 'twitter', 'is', 'a', 'good', '<END>']
['<START>', '@spongebobbway', 'yes', 'i', 'love', 'my', 'life', '<END>']
['<START>', 'what’s', 'is', 'a', 'one', 'on', 'my', 'life', '<END>']
['<START>', '@agcook404…', 'ate', 'omg', '<END>']
['<START>', 'people:', 'hate', 'my', 'life', '<END>']
['<START>', 'give', 'me', 'in', 'the', 'robyn', 'concert', 'concert', '<END>']
['<START>', 'let', 'do', 'me', '<END>']
['<START>', 'omg', 'yes', '<END>']


Some of my test results:
Semi-beam with lstm length 256 trained for 5 epochs with accuracy = 21%

TODO: try making the model more compicated eg
model2.add(Dropout(0.2))
model2.add(LSTM(128))
model2.add(Dropout(0.2))

